In [22]:
%pip install Faker


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Imports for generate fake data and import util libs
from faker import Factory
from faker import Faker
import random

import pandas as pd
import numpy as np

import datetime
from datetime import timedelta
import csv

fake = Factory.create('es_MX')

number_of_records = 75000

In [23]:
def get_random_category() -> list:
  categories = ['R1', 'R2', 'R3', 'R4', 'C', 'CE', 'I', 'P', 'S', 'I']
  return random.choice(categories)

def get_random_zone() -> str:
  zones = ['N', 'S', 'NE', 'NO', 'E', 'O', 'SE', 'SO', 'C']
  return random.choice(zones)

def generate_meter_code() -> str:
  code = ['']*10
  code[0] = chr(random.randrange(65, 90))
  code[1] = str(random.randrange(1, 9))
  code[2] = chr(random.randrange(65, 90))
  code[3] = chr(random.randrange(65, 90))
  for i in [4, 5, 6, 7, 8, 9]: code[i] = str(random.randrange(1, 9))
  return ''.join(code)

def get_random_date() -> str:
  start_date = datetime.date(2021, 1, 1)
  end_date   = datetime.date(2023, 12, 31)
  num_days   = (end_date - start_date).days
  rand_days   = random.randint(1, num_days)
  random_date = start_date + datetime.timedelta(days=rand_days)
  return str(random_date)

In [19]:
# categories_range = {
#     'R1': 20.04401,
#     'R2': 39.95963,
#     'R3': 74.90816,
#     'R4': 124.77741,
#     'C': 149.84842,
#     'CE': 174.77506,
#     'I': 134.86357,
#     'P': 65.80169,
#     'S': 109.80858,
# }
def get_consumption_value(category:str=None)->list:
  categories_range = {
    'R1': (0,25),
    'R2': (0,25),
    'R3': (0,25),
    'R4': (0,25),
    'C': (13,100),
    'CE': (50, 250),
    'I': (76, 500),
    'P': (200, 500),
    'S': (50, 200),
  }
  return random.randint(
    categories_range[category][0],
    categories_range[category][1]
  )


In [24]:
# create names for users
df_users = pd.DataFrame({
    'id' : np.arange(number_of_records),
    'name' : [fake.name() for x in range(number_of_records)],
    'address' : [fake.address() for x in range(number_of_records)]
})
df_users.to_csv('../dataset/users.csv', index=False)
df_users.set_index('id')

,name,address
id,,
0,Antonio Emiliano Vázquez Saucedo,"Calle Flórez 547 Edif. 110 , Depto. 669\nSan S..."
1,Mercedes Arriaga,"Calle Sur Avilés 732 596\nVieja Lituania, MOR ..."
2,Serafín Lomeli Lebrón,"Circuito Sur Ibarra 984 Edif. 103 , Depto. 141..."
3,Dulce Blanco,Calzada Etiopía 942 Interior 101\nVieja Tuvalu...
4,Lic. Bernardo Sevilla,Boulevard Santo Tomé y Príncipe 235 Interior 8...
...,...,...
74995,Elias Maldonado Sevilla,Periférico Durango 646 Interior 201\nNueva Arg...
74996,Isabel Mauro Narváez Lara,"Ampliación Norte Núñez 818 997\nNueva Libia, B..."
74997,Linda Lilia Gracia Barrientos,"Avenida Guanajuato 560 Edif. 198 , Depto. 137\..."


In [25]:
df_meters  = pd.DataFrame({
    'id' : np.arange(number_of_records),
    'meter_code' : [generate_meter_code() for x in range(number_of_records)],
    'category' : [get_random_category() for x in range(number_of_records)],
    'zone': [get_random_zone() for x in range(number_of_records)],
    'date_creation':[get_random_date() for x in range(number_of_records)]
})
df_meters.to_csv('../dataset/meters.csv', index=False)
df_meters.set_index('id')

,meter_code,category,zone,date_creation
id,,,,
0,A7XJ718334,S,SE,2021-07-13
1,D8JU555818,R4,NE,2021-02-05
2,E3GM687457,I,SE,2023-02-16
3,X3CU414126,S,E,2022-05-14
4,U7CD525161,R2,O,2023-09-16
...,...,...,...,...
74995,V5CH721872,I,N,2022-12-29
74996,N2RJ318631,C,SO,2023-03-28
74997,N3FT433128,C,SE,2023-10-16


In [26]:
df_meters['year'] = df_meters['date_creation'].apply(lambda y: int(y.split('-')[0]))
df_meters['month'] = df_meters['date_creation'].apply(lambda y: int(y.split('-')[1]))

In [27]:
end_year = 2023
end_month = 12
months = ['ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dic']

columns = ['meter_code', 'period', 'lecture']

with open('../dataset/lectures.csv', 'w', newline='') as file:

  writer = csv.writer(file)
  writer.writerow(columns)
  for i in range(len(df_meters)):

    start_year = df_meters.loc[i, 'year']
    start_month = df_meters.loc[i, 'month']

    for m in range(start_month, end_month+1):
      writer.writerow([
        df_meters.loc[i, 'meter_code'],
        f'{months[m-1]}-{start_year}',
        get_consumption_value(df_meters.loc[i, 'category'])]
      )

    for y in range(start_year+1, end_year+1):
      for m in range(1, end_month+1):
        writer.writerow([
          df_meters.loc[i, 'meter_code'],
          f'{months[m-1]}-{y}',
          get_consumption_value(df_meters.loc[i, 'category'])
        ])

In [28]:
df_lectures = pd.read_csv('../dataset/lectures.csv')
df_lectures

,meter_code,period,lecture
0,A7XJ718334,jul-2021,156
1,A7XJ718334,ago-2021,102
2,A7XJ718334,sep-2021,104
3,A7XJ718334,oct-2021,66
4,A7XJ718334,nov-2021,72
...,...,...,...
1384497,X7OC137648,ago-2023,235
1384498,X7OC137648,sep-2023,136
1384499,X7OC137648,oct-2023,188
1384500,X7OC137648,nov-2023,227
